<a href="https://colab.research.google.com/github/anitaainandiyah/ainandiyah/blob/master/Srapping_Google_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing libraries and packages in ubuntu
!sudo apt update
!sudo apt-get upgrade
!apt install chromium-chromedriver
!pip install selenium
!pip install selenium-wire
!sudo apt install openssl
!pip install --upgrade requests

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,861 kB]
Get:14 http://archive.ubuntu.

In [2]:
import pandas as pd
import requests,json,time,random
from selenium.webdriver.common.by import By
import sys
from seleniumwire.utils import decode
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from seleniumwire import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=options)
timeout = 5

In [14]:
def round( n ):
    a = (n // 10) * 10
    b = a + 10
    return (a if a == n else b)


#Get the google maps api url
def getGoogleMapsApiURL(url):
  driver = webdriver.Chrome('chromedriver',options=options)
  site = driver.get(url)
  time.sleep(4)
  
 
  # Get total number of reviews

  number_of_reviews = driver.find_element(by=By.CSS_SELECTOR,value='div[jsan="7.fontBodySmall,0.jslog"]').text  

  number_of_reviews = int(''.join(filter(str.isdigit, number_of_reviews)))
  number_of_calls = round(number_of_reviews)
  print(f"Getting Url...")
  # driver.save_screenshot("image.png")
  # Sort by recent
  sort_btn = driver.find_element(by=By.CSS_SELECTOR,value='button[aria-label="Sort reviews"]')
  sort_btn.click()
  time.sleep(random.uniform(1, 2))
  newest = driver.find_element(by=By.CSS_SELECTOR,value='ul[role = "menu"] li:nth-child(2)')
  newest.click()
  time.sleep(random.uniform(1, 3))

  #Find scroll layout
  place_rating = driver.find_element(by=By.CLASS_NAME,value="fontDisplayLarge").text
  try:
    scrollable_div = driver.find_element(by=By.CSS_SELECTOR, value = 'div[data-review-id] div[data-review-id]')
  except:
    scrollable_div = driver.find_element(by=By.CSS_SELECTOR, value = 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')

  

  #Scroll Once
  last_element = ""
  for i in range(0,2):
          driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
                  scrollable_div)

  #Get the api url from driver logs 
  api_url = "not found"
  headers = "not found"
  for item in driver.requests:
       if "listentitiesreviews" in str(item):
        api_url = str(item)
        headers = dict(item.headers)  
        headers = (headers)  
        break
  driver.find_element(by=By.CSS_SELECTOR,value='button[aria-label="Back"]').click()
  time.sleep(random.uniform(1, 2))
  place_info = {}
  place_name = driver.find_element(by=By.TAG_NAME,value = "h1").text
  place_info["Place Name"] = place_name
  print(f"Strating crawling for {place_name}")
  print(f"{number_of_reviews} reviews has been found.")
  place_info["Place Rating"] = place_rating
  for el in ["Copy address","Open website","Copy phone number","Copy plus code",'button[jsaction="pane.rating.category"]','div[role="region"][aria-label^="About"]']:
    try:
      if "Copy" in el:
        place_info[el.split("Copy")[1].title()] = driver.find_element(by=By.CSS_SELECTOR,value=f'button[data-tooltip="{el}"]').text
      elif "category" in el:
        place_info['Category'] = driver.find_element(by=By.CSS_SELECTOR,value=el).text
      elif "About" in el:
        place_info["About"] = driver.find_element(by=By.CSS_SELECTOR,value=el).text
      else:
        web_info = driver.find_elements(by=By.CSS_SELECTOR,value=f'button[data-tooltip="{el}"]')[0].get_attribute("aria-label")
        place_info[el.split("Open")[1].title()] = web_info.split("Website: ")[1]
    except:
      pass
  return api_url,number_of_calls,headers,place_info

#Get Review df
def getGoogleMapsReviews(url,date_limit="Unlimited"):
  api_url,number_of_calls,HEADERS,place_info = getGoogleMapsApiURL(url)
  print(place_info)
  if api_url == "not found":
    print("Could not fetch api url :( , try after sometime")
  else:
    df_data_list = []
    last_item = []
    j = 0
    for call in range(0,number_of_calls,10):
      response_data = requests.get(api_url.replace("!1i0!",f"!1i{call}!"),headers=HEADERS)
      # print(response_data.text)
      try:
        response_json = response_data.text.split(")]}'")[1]
      except:
        j = 1
      if j == 1:
        break
      print(f"\rCrawling {int(len(df_data_list)/10)+1} page{'.'*random.randint(2,4)}",end="")
      for data in json.loads(response_json)[2]:
        # print(data)
        try:
          author = data[0][1]
          date = data[27]
          date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(date/1000.0))
          if date_limit != "Unlimited" and date_limit > date:
            print(f"\nCurrent review date : {date} ***** User set date limit : {date_limit}")
            j = 1
            print("Date set has been reached")
            break

          review = data[3]
          rating = data[4]
          current_item = [author,date,review,rating]
          df_data_list.append(current_item)
        except Exception as e:
          print(e)
          print(f"Loaded {len(df_data_list)} reviews but got captcha issues")
          j = 1
      time.sleep(random.uniform(1,3))
    df = pd.DataFrame(df_data_list,columns=["Author","date","Review","Review Rating"])
    if df.shape[0] > 0:
      df['Url'] = url
      for key in place_info:
        df[key] = place_info[key]
      print(f"{df.shape[0]} reviews crawled")
      print(f"Crawl Completed for {place_info['Place Name']}\n")

      return df 
    else:
      print("No reviews crawled for",url)

In [15]:
df = getGoogleMapsReviews(url = "https://www.google.com/maps/place/Wisata+Bahari+Lamongan+(WBL)/@-6.8664183,112.3574465,17z/data=!4m7!3m6!1s0x2e77c3505a57d187:0x1eb952764236c3c8!8m2!3d-6.8664236!4d112.3596352!9m1!1b1")
df

Getting Url...
Strating crawling for Wisata Bahari Lamongan (WBL)
1752 reviews has been found.
{'Place Name': 'Wisata Bahari Lamongan (WBL)', 'Place Rating': '4.5', ' Address': 'Wisata bahari, Paciran, Kec. Paciran, Kabupaten Lamongan, Jawa Timur 62276, Indonesia', ' Website': 'wisatabaharilamongan.com ', ' Plus Code': '49M5+CV Paciran, Lamongan Regency, East Java, Indonesia', 'Category': 'Tourist attraction'}
Crawling 176 page....1752 reviews crawled
Crawl Completed for Wisata Bahari Lamongan (WBL)



,Author,date,Review,Review Rating,Url,Place Name,Place Rating,Address,Website,Plus Code,Category
0,Rofik_ ngrao,2022-06-22 22:11:11,None,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
1,「 SHM 」 2727,2022-06-22 11:31:05,None,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
2,Nurdiffa Udin,2022-06-22 10:24:30,None,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
3,Ery Isdhianti,2022-06-22 09:53:21,(Translated by Google) Extraordinary\n\n(Origi...,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
4,Ay'N Witantri,2022-06-22 07:18:52,None,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
...,...,...,...,...,...,...,...,...,...,...,...
1747,Sandy Marsena,2021-05-15 13:27:37,(Translated by Google) The roller coaster is c...,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
1748,azka rosa,2021-05-15 03:54:58,(Translated by Google) really good\n\n(Origina...,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
1749,Nafisatul Wardah,2021-05-10 18:11:17,None,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction
1750,Romi Hanif,2021-05-08 21:14:46,(Translated by Google) The place is exciting. ...,5,https://www.google.com/maps/place/Wisata+Bahar...,Wisata Bahari Lamongan (WBL),4.5,"Wisata bahari, Paciran, Kec. Paciran, Kabupate...",wisatabaharilamongan.com,"49M5+CV Paciran, Lamongan Regency, East Java, ...",Tourist attraction


In [17]:
df.to_csv('google_user_reviews.csv')